# 1. Nearest Neighbor

In [1]:
from lab_tools import CIFAR10, get_hog_image

# Load Dataset
dataset = CIFAR10('CIFAR10/CIFAR10/')

ModuleNotFoundError: No module named 'numpy'

In [2]:
from lab_tools import CIFAR10, get_hog_image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
import numpy as np

# Load Dataset
dataset = CIFAR10('CIFAR10/CIFAR10/')

# 1. Fraction of data for validation
p_val = 0.1
n_val = int(p_val * len(dataset.train['hog']))

# 2. Training and Validation Split
train_X = dataset.train['hog'][:-n_val]
train_Y = dataset.train['labels'][:-n_val]

val_X = dataset.train['hog'][-n_val:]
val_Y = dataset.train['labels'][-n_val:]

print(train_X.shape, train_Y.shape, val_X.shape, val_Y.shape)

# Descriptive Performance
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(train_X, train_Y)
pred_descr = clf.predict(train_X)
descriptive_score = accuracy_score(train_Y, pred_descr)
print(f"Descriptive Score: {descriptive_score:.3f}")

# Predictive Performance using Cross-Validation
k_values = [1, 3, 4, 5, 6, 7, 9]
cv_scores = []
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(clf, train_X, train_Y, cv=10)
    cv_scores.append(scores.mean())
    print(f"K={k}, Cross-Validation Score: {scores.mean():.3f}")

# Best K based on Cross-Validation
best_k_index = np.argmax(cv_scores)
best_k = k_values[best_k_index]
print(f"Best K based on Cross-Validation: {best_k}")

# Final Predictive Score using Best K
clf = KNeighborsClassifier(n_neighbors=best_k)
clf.fit(train_X, train_Y)
pred = clf.predict(val_X)
final_score = accuracy_score(val_Y, pred)
print(f"Predictive Score using Best K: {final_score:.3f}")

# Confusion Matrix for Best K
cm = confusion_matrix(val_Y, pred)
print("Confusion Matrix for Best K:\n", cm)
print("Sum by Class:", cm.sum(axis=1))


ModuleNotFoundError: No module named 'numpy'

In [7]:
from lab_tools import CIFAR10, get_hog_image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Load Dataset
dataset = CIFAR10('CIFAR10/CIFAR10/')

# 1. Fraction of data for validation
p_val = 0.1
n_val = int(p_val * len(dataset.train['hog']))

# 2. Training and Validation Split
train_X = dataset.train['hog'][:-n_val]
train_Y = dataset.train['labels'][:-n_val]

val_X = dataset.train['hog'][-n_val:]
val_Y = dataset.train['labels'][-n_val:]

print(train_X.shape, train_Y.shape, val_X.shape, val_Y.shape)

# Descriptive Performance
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(train_X, train_Y)
pred_descr = clf.predict(train_X)
descriptive_score = accuracy_score(train_Y, pred_descr)
print(f"Descriptive Score: {descriptive_score:.3f}")

# Predictive Performance using Stratified Cross-Validation and Different Metrics
k_values = [1, 3, 5, 7, 9]
metrics = ['euclidean', 'manhattan', 'chebyshev']
skf = StratifiedKFold(n_splits=5)

for metric in metrics:
    print(f"\nMetric: {metric}")
    cv_scores = []
    for k in k_values:
        clf = KNeighborsClassifier(n_neighbors=k, metric=metric)
        scores = cross_val_score(clf, train_X, train_Y, cv=skf)
        cv_scores.append(scores.mean())
        print(f"K={k}, Cross-Validation Score: {scores.mean():.3f}")

    # Best K based on Cross-Validation for this metric
    best_k_index = np.argmax(cv_scores)
    best_k = k_values[best_k_index]
    print(f"Best K for {metric}: {best_k}")

    # Final Predictive Score using Best K for this metric
    clf = KNeighborsClassifier(n_neighbors=best_k, metric=metric)
    clf.fit(train_X, train_Y)
    pred = clf.predict(val_X)
    final_score = accuracy_score(val_Y, pred)
    print(f"Predictive Score using Best K ({metric}): {final_score:.3f}")

    # Confusion Matrix for Best K and Metric
    cm = confusion_matrix(val_Y, pred)
    print("Confusion Matrix for Best K ({metric}):\n", cm)
    print("Sum by Class:", cm.sum(axis=1))


Pre-loading training data
Pre-loading test data
(13500, 256) (13500,) (1500, 256) (1500,)
Descriptive Score: 0.805

Metric: euclidean
K=1, Cross-Validation Score: 0.683
K=3, Cross-Validation Score: 0.700
K=5, Cross-Validation Score: 0.704
K=7, Cross-Validation Score: 0.701
K=9, Cross-Validation Score: 0.699
Best K for euclidean: 5
Predictive Score using Best K (euclidean): 0.713
Confusion Matrix for Best K ({metric}):
 [[330 129  53]
 [ 35 388  67]
 [ 15 131 352]]
Sum by Class: [512 490 498]

Metric: manhattan
K=1, Cross-Validation Score: 0.741
K=3, Cross-Validation Score: 0.762
K=5, Cross-Validation Score: 0.770
K=7, Cross-Validation Score: 0.768
K=9, Cross-Validation Score: 0.768
Best K for manhattan: 5
Predictive Score using Best K (manhattan): 0.769
Confusion Matrix for Best K ({metric}):
 [[393  89  30]
 [ 42 390  58]
 [ 15 112 371]]
Sum by Class: [512 490 498]

Metric: chebyshev
K=1, Cross-Validation Score: 0.589
K=3, Cross-Validation Score: 0.615
K=5, Cross-Validation Score: 0.6

In [8]:
from lab_tools import CIFAR10, get_hog_image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

# Load Dataset
dataset = CIFAR10('CIFAR10/CIFAR10/')

# 1. Fraction of data for validation
p_val = 0.1
n_val = int(p_val * len(dataset.train['hog']))

# 2. Training and Validation Split
train_X = dataset.train['hog'][:-n_val]
train_Y = dataset.train['labels'][:-n_val]

val_X = dataset.train['hog'][-n_val:]
val_Y = dataset.train['labels'][-n_val:]

print(train_X.shape, train_Y.shape, val_X.shape, val_Y.shape)

# Preprocessing: Normalization
scaler = StandardScaler()
train_X_norm = scaler.fit_transform(train_X)
val_X_norm = scaler.transform(val_X)

# Preprocessing: PCA (95% Variance Retention)
pca = PCA(n_components=0.95)
train_X_pca = pca.fit_transform(train_X_norm)
val_X_pca = pca.transform(val_X_norm)

# Descriptive Performance - Original
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(train_X, train_Y)
pred_descr = clf.predict(train_X)
descriptive_score = accuracy_score(train_Y, pred_descr)
print(f"Descriptive Score (Original): {descriptive_score:.3f}")

# Predictive Performance using Stratified Cross-Validation and Different Metrics
k_values = [1, 3, 5, 7, 9]
metrics = ['euclidean', 'manhattan', 'chebyshev']
skf = StratifiedKFold(n_splits=5)

for metric in metrics:
    print(f"\nMetric: {metric}")
    for data, label in zip(['Original', 'Normalized', 'PCA'], [train_X, train_X_norm, train_X_pca]):
        print(f"\nData: {data}")
        cv_scores = []
        for k in k_values:
            clf = KNeighborsClassifier(n_neighbors=k, metric=metric)
            scores = cross_val_score(clf, label, train_Y, cv=skf)
            cv_scores.append(scores.mean())
            print(f"K={k}, Cross-Validation Score: {scores.mean():.3f}")

        # Best K based on Cross-Validation for this metric and data
        best_k_index = np.argmax(cv_scores)
        best_k = k_values[best_k_index]
        print(f"Best K for {metric} ({data}): {best_k}")

        # Final Predictive Score using Best K
        clf = KNeighborsClassifier(n_neighbors=best_k, metric=metric)
        clf.fit(label, train_Y)
        if data == 'Original':
            pred = clf.predict(val_X)
        elif data == 'Normalized':
            pred = clf.predict(val_X_norm)
        else:
            pred = clf.predict(val_X_pca)

        final_score = accuracy_score(val_Y, pred)
        print(f"Predictive Score using Best K ({metric}, {data}): {final_score:.3f}")

        # Confusion Matrix
        cm = confusion_matrix(val_Y, pred)
        print("Confusion Matrix for Best K ({metric}, {data}):\n", cm)
        print("Sum by Class:", cm.sum(axis=1))


Pre-loading training data
Pre-loading test data
(13500, 256) (13500,) (1500, 256) (1500,)
Descriptive Score (Original): 0.805

Metric: euclidean

Data: Original
K=1, Cross-Validation Score: 0.683
K=3, Cross-Validation Score: 0.700
K=5, Cross-Validation Score: 0.704
K=7, Cross-Validation Score: 0.701
K=9, Cross-Validation Score: 0.699
Best K for euclidean (Original): 5
Predictive Score using Best K (euclidean, Original): 0.713
Confusion Matrix for Best K ({metric}, {data}):
 [[330 129  53]
 [ 35 388  67]
 [ 15 131 352]]
Sum by Class: [512 490 498]

Data: Normalized
K=1, Cross-Validation Score: 0.683
K=3, Cross-Validation Score: 0.708
K=5, Cross-Validation Score: 0.716
K=7, Cross-Validation Score: 0.713
K=9, Cross-Validation Score: 0.714
Best K for euclidean (Normalized): 5
Predictive Score using Best K (euclidean, Normalized): 0.729
Confusion Matrix for Best K ({metric}, {data}):
 [[352 120  40]
 [ 43 378  69]
 [ 16 119 363]]
Sum by Class: [512 490 498]

Data: PCA
K=1, Cross-Validation 

# 2. Decision Trees


In [9]:
from lab_tools import CIFAR10, get_hog_image
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Load Dataset
dataset = CIFAR10('CIFAR10/CIFAR10/')

# 1. Fraction of data for validation
p_val = 0.1
n_val = int(p_val * len(dataset.train['hog']))

# 2. Training and Validation Split
train_X = dataset.train['hog'][:-n_val]
train_Y = dataset.train['labels'][:-n_val]

val_X = dataset.train['hog'][-n_val:]
val_Y = dataset.train['labels'][-n_val:]

print(train_X.shape, train_Y.shape, val_X.shape, val_Y.shape)

# Descriptive Performance - Decision Tree
clf = DecisionTreeClassifier(random_state=42)
clf.fit(train_X, train_Y)
pred_descr = clf.predict(train_X)
descriptive_score = accuracy_score(train_Y, pred_descr)
print(f"Descriptive Score (Decision Tree): {descriptive_score:.3f}")

# Predictive Performance using Stratified Cross-Validation
skf = StratifiedKFold(n_splits=5)
max_depth_values = [5, 10, 15, 20, None]
cv_scores = []

for depth in max_depth_values:
    clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
    scores = cross_val_score(clf, train_X, train_Y, cv=skf)
    cv_scores.append(scores.mean())
    print(f"Max Depth={depth}, Cross-Validation Score: {scores.mean():.3f}")

# Best Max Depth based on Cross-Validation
best_depth_index = np.argmax(cv_scores)
best_depth = max_depth_values[best_depth_index]
print(f"Best Max Depth based on Cross-Validation: {best_depth}")

# Final Predictive Score using Best Max Depth
clf = DecisionTreeClassifier(max_depth=best_depth, random_state=42)
clf.fit(train_X, train_Y)
pred = clf.predict(val_X)
final_score = accuracy_score(val_Y, pred)
print(f"Predictive Score using Best Max Depth: {final_score:.3f}")

# Confusion Matrix for Best Depth
cm = confusion_matrix(val_Y, pred)
print("Confusion Matrix for Best Max Depth:\n", cm)
print("Sum by Class:", cm.sum(axis=1))

Pre-loading training data
Pre-loading test data
(13500, 256) (13500,) (1500, 256) (1500,)
Descriptive Score (Decision Tree): 1.000
Max Depth=5, Cross-Validation Score: 0.578
Max Depth=10, Cross-Validation Score: 0.593
Max Depth=15, Cross-Validation Score: 0.582
Max Depth=20, Cross-Validation Score: 0.573
Max Depth=None, Cross-Validation Score: 0.570
Best Max Depth based on Cross-Validation: 10
Predictive Score using Best Max Depth: 0.604
Confusion Matrix for Best Max Depth:
 [[325 122  65]
 [ 91 307  92]
 [ 62 162 274]]
Sum by Class: [512 490 498]
